In [1]:
import torch
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np

import torchinfo
import importlib as imp

import data_loader.data_loader as data_loader
from trainer.trainer import Trainer
from model.model import TorchModel
from utils import utils
from model.loss import GaussianNLL, ShashNLL

torch.set_warn_always(False)

# https://github.com/victoresque/pytorch-template/tree/master

# TODO: add a logger
# TODO: add metrics
# TODO: change from settings to config files

In [2]:
settings = {
    "batch_size": 32,
    "device": "gpu",
    "max_epochs": 10_000,
    "learning_rate": 0.00001,
    "patience": 2,
    "min_delta": 0.05,
    "criterion": ShashNLL(),
}

In [3]:
trainset = data_loader.CustomData("data/train_data.pickle")
valset = data_loader.CustomData("data/val_data.pickle")
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=settings["batch_size"], shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    valset, batch_size=settings["batch_size"], shuffle=False
)

In [4]:
SEED = 44
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True


trainset = data_loader.CustomData("data/train_data.pickle")
valset = data_loader.CustomData("data/val_data.pickle")
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=settings["batch_size"], shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    valset, batch_size=settings["batch_size"], shuffle=False
)

model = TorchModel(target=trainset.target)
model.freeze_layers(freeze_id="tau")
device = utils.prepare_device(settings["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=settings["learning_rate"])

trainer = Trainer(
    model,
    settings["criterion"],
    optimizer,
    max_epochs=settings["max_epochs"],
    device=device,
    data_loader=train_loader,
    validation_data_loader=val_loader,
    settings=settings,
)

torchinfo.summary(
    model,
    [
        trainset.input[: settings["batch_size"]].shape,
        trainset.input_unit[: settings["batch_size"]].shape,
    ],
    verbose=0,
)

/opt/homebrew/Caskroom/miniforge/base/envs/env-torch-sonoma/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Layer (type:depth-idx)                   Output Shape              Param #
TorchModel                               [32, 4]                   --
├─CircularPad2d: 1-1                     [32, 1, 72, 154]          --
├─Sequential: 1-2                        [32, 5760]                --
│    └─Sequential: 2-1                   [32, 32, 36, 77]          --
│    │    └─Conv2d: 3-1                  [32, 32, 72, 154]         832
│    │    └─ReLU: 3-2                    [32, 32, 72, 154]         --
│    │    └─MaxPool2d: 3-3               [32, 32, 36, 77]          --
│    └─Sequential: 2-2                   [32, 32, 18, 39]          --
│    │    └─Conv2d: 3-4                  [32, 32, 36, 77]          9,248
│    │    └─ReLU: 3-5                    [32, 32, 36, 77]          --
│    │    └─MaxPool2d: 3-6               [32, 32, 18, 39]          --
│    └─Sequential: 2-3                   [32, 32, 9, 20]           --
│    │    └─Conv2d: 3-7                  [32, 32, 18, 39]          9,248
│    │  

In [5]:
# raise ValueError()

In [6]:
model = model.to(device)
trainer.fit()

Epoch   0/10000
  10.5s - train_loss: 1.10351 - val_loss: 0.95180
Epoch   1/10000
  10.0s - train_loss: 0.82079 - val_loss: 0.69008
Epoch   2/10000
  10.1s - train_loss: 0.63215 - val_loss: 0.54606
Epoch   3/10000
  10.0s - train_loss: 0.52397 - val_loss: 0.44674
Epoch   4/10000
  10.3s - train_loss: 0.44095 - val_loss: 0.37889


KeyboardInterrupt: 

In [ ]:
input = torch.Tensor(valset.input[:3]).to(device)
input_unit = torch.Tensor(valset.input_unit[:3]).to(device)
with torch.no_grad():
    pred = model(input, input_unit)
pred = pred.cpu().numpy()
pred

In [ ]:
from shash.shash_torch import Shash

sample = 1
x = np.arange(-13, 13, 0.01)
dist = Shash(
    mu=pred[sample, 0],
    sigma=pred[sample, 1],
    gamma=pred[sample, 2],
    tau=pred[sample, 3],
)
p = dist.prob(torch.tensor(x))

plt.plot(x, p)